In [1]:
import pandas as pd
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
from numpy import where
import numpy as np
import statistics 


In [2]:
df = pd.read_csv("../dataset/DATA_FINAL_221.csv",dtype = None)
df = df.drop(["ALARM"],axis = 1)

C:\Users\A2IN\AppData\Local\Temp\ipykernel_17364\1419856285.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../dataset/DATA_FINAL_221.csv",dtype = None)


In [3]:
df = df.dropna()


In [4]:
df = df.astype(float)

In [5]:
df["mean"] = df.iloc[:,1:4].mean(axis = 1)

In [6]:
df

,RESP,ABP1,ABP2,ABP3,SPO2,HR,PULSE,mean
0,23.0,118.0,182.0,80.0,99.0,75.0,75.0,126.666667
1,23.0,118.0,182.0,80.0,99.0,76.0,75.0,126.666667
2,22.0,118.0,182.0,80.0,99.0,76.0,76.0,126.666667
3,22.0,118.0,182.0,80.0,99.0,76.0,76.0,126.666667
4,22.0,118.0,183.0,80.0,99.0,76.0,76.0,127.000000
...,...,...,...,...,...,...,...,...
83353,21.0,109.0,185.0,74.0,98.0,82.0,81.0,122.666667
83354,21.0,109.0,184.0,74.0,98.0,82.0,81.0,122.333333
83355,21.0,110.0,185.0,74.0,98.0,82.0,81.0,123.000000
83356,21.0,110.0,186.0,75.0,98.0,82.0,81.0,123.666667


In [7]:
df = df.drop(["ABP1","ABP2","ABP3"],axis = 1)

In [8]:
df

,RESP,SPO2,HR,PULSE,mean
0,23.0,99.0,75.0,75.0,126.666667
1,23.0,99.0,76.0,75.0,126.666667
2,22.0,99.0,76.0,76.0,126.666667
3,22.0,99.0,76.0,76.0,126.666667
4,22.0,99.0,76.0,76.0,127.000000
...,...,...,...,...,...
83353,21.0,98.0,82.0,81.0,122.666667
83354,21.0,98.0,82.0,81.0,122.333333
83355,21.0,98.0,82.0,81.0,123.000000
83356,21.0,98.0,82.0,81.0,123.666667


In [9]:
def lisa(k,cur):
    var = 0
    mean = cur.iloc[-1].mean(axis = 0)
    std = statistics.stdev(cur.iloc[-1])
    for i in cur:
        corr = 0
        z = 0
        if i != k:
            corr = cur[k].corr(cur[i])
            z =  cur[i].iat[-1]
            z = z - mean
            if std != 0 :
                z /= std
            var += (z*corr)
    z =  cur[k].iat[-1]
    z = z - mean
    if std != 0 :
        z /= std
    var = (var*z)
    
    return var

In [10]:
a = len(df)
w = 100
thres = 0.75
y_pred = np.zeros(w-1)
for i in range(a-w+1):
    cur = df.iloc[i:i+w]
    cou = np.zeros(0)
    for k in cur:
        l = lisa(k,cur)
        cou = np.append(cou, l)
    y_pred = np.append(y_pred,cou)

In [11]:
print(y_pred)


[ 0.          0.          0.         ...  0.00133431  0.00298618
 -0.12290197]


In [12]:
df1 = pd.read_csv("../dataset/DATA_FINAL_221.csv")


C:\Users\A2IN\AppData\Local\Temp\ipykernel_17364\2804527451.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("../dataset/DATA_FINAL_221.csv")


In [13]:
len(y_pred)

415544

In [14]:
df1 = df1.dropna()

In [15]:
act = []
for i in df1["ALARM"]:
    if i == False:
        act.append(0)
    else :
        act.append(1)

In [16]:
len(act)

83188

In [17]:
print(len(y_pred),len(act))

415544 83188


In [18]:
i = 0
tp =tn =fp =fn = 0

for i in range(len(y_pred)):
    if act[i] == y_pred[i]:
        if act[i] == 0:
            tn +=1
        else :
            tp +=1
    else :
        if y_pred[i] == 0:
            fn += 1
        else :
            fp += 1

IndexError: list index out of range

In [ ]:
tral = 0
faal = 0

for i in range(len(act)):
    if act[i] == 0:
        tral +=1
    else :
        faal +=1
print(tral, faal)

In [91]:
tral = 0
faal = 0

for i in range(len(y_pred)):
    if y_pred[i] == 0:
        tral +=1
    else :
        faal +=1
print(tral, faal)

82989 199


In [92]:
print(tp,tn,fp,fn)

5 80913 194 2076


In [93]:
accuracy = (tp+tn)/(tp+tn+fn+fp)
tpr = tp/(tp+fn)
far = fp/(fp+tn)

In [94]:
print(accuracy,tpr,far)

0.972712410443814 0.002402691013935608 0.0023919020553096527


0.557625274183401 0.5557844013523491 0.4388590653835688 ----- delta = 0.1 w = 100---------------------
0.5842359422557004 0.6207593362608324 0.48551567177102145 --- delta = 0.4    w = 100-------------------------
0.5243151791331554 0.47197502558323295 0.3757266902505999  --- delta = 0.4   w= 50-----------------
0.5043444253626023 0.41907278591691605 0.3328055809811246   --- delta = 0.1   w= 50-----------------
0.6281733009130945 0.7334421430329409 0.5728669321920689   --- delta = 0.4   w= 200-----------------
0.6427964156365306 0.7802432673998367 0.6196967073200901  --- delta = 0.4   w= 300-----------------
0.648917719474248 0.806318734698636 0.6516834475422408   --- delta = 0.4   w= 400-----------------



0.15208924364090975 0.9702066314271984 0.8689015744633632  ---- delta = 0.4 w = 400 cou = 1 ---------
different technique at threshold, instead of lisa < threshold, it is made into lisa > threshold and threshold is significantly increased, now #of cous also increased to 3 instead of 1 for the older method